# Validation

Documentation des librairies utilisées dans ce tp:
- scikit-learn: [https://scikit-learn.org/stable/index.html](https://scikit-learn.org/stable/index.html)
- matplotlib: [https://matplotlib.org/](https://matplotlib.org/)
- numpy: [https://numpy.org/](https://numpy.org/)

Instructions d'installation dans un environnement conda, si vous voulez utiliser votre propre ordinateur (inutile de suivre ces instructions sur les machines de la salle info).

- Créer un nouvel environnement, appelé `sd3` (vous avez besoin d'avoir anaconda ou miniconda installé)
```
conda create -n sd3 python=3.9
```
- Activer l'environnement
```
conda activate sd3
```
- Installer les librairies requises
```
conda install -c conda-forge jupyterlab matplotlib scikit-learn
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.datasets import load_digits, make_classification, fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.utils import shuffle

### Données MNIST

On va utiliser ici le jeu de données MNIST, qui contient des images de chiffres écrits à la main, représentés par des images en nuances de gris de 28x28 pixels.
On peut obtenir ces données grâce au code écrit dans la cellule suivante.
L'obtention du jeu de données peut prendre du temps: il est conseillé de n'exécuter ce code qu'une fois.

In [ ]:
mnist = fetch_openml("mnist_784", version=1, as_frame=False)

In [ ]:
X = mnist.data
Y = mnist.target
# On prend un sous-ensemble des données pour accélérer les calculs et rendre le tp plus rapide.
# Dans une application réelle, on garderait toutes les données
X,Y = shuffle(X,Y)
X = X[:10000]
Y = Y[:10000]

- Combien de données sont présentes dans mnist ? Combien de classes ? Afficher quelques exemples à l'aide de la fonction `plot_examples` définie ci-dessous.

In [ ]:
# Cette fonction affiche 4 exemples issus du dataset
def plot_examples(data, labels):  # données, étiquettes
    _, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
    nb_pxl = len(data[0].flatten())
    side_len = int(np.sqrt(nb_pxl))  # taille du côté de l'image (supposée carrée)
    for ax, image, label in zip(axes, data, labels):
        ax.set_axis_off()
        image = image.reshape((side_len, side_len))
        ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
        ax.set_title("Label: " + str(label))

- Séparer l'ensemble de données en trois ensembles: `X_train` avec étiquettes `Y_train` (60% des exemples), `X_val` et `Y_val` (20% des exemples), `X_test` et `Y_test` (20% des exemples). On pourra utiliser la fonction `train_test_split`.

L'ensemble de validation va servir à choisir les hyper-paramètres de nos classeurs.

### Le perceptron multi-couches

Le perceptron multi-couches (MLP), ou réseau de neurones complètement connecté, est constitué de plusieurs couches de neurones.

In [ ]:
mlp = MLPClassifier(
    hidden_layer_sizes=(10,), activation="relu",
    solver="sgd", learning_rate="constant", learning_rate_init=0.001,
    max_iter=2000, verbose=False)

`hidden_layer_sizes=(15,10,)` indique le nombre de neurones dans chaque couche cachée du réseau de neurones. Ici deux couches cachées, avec respectivement 15 et 10 neurones.

`activation` décide la fonction d'activation de chaque neurone.

`solver` indique la méthode d'apprentissage des poids. "sgd" = descente de gradient stochastique.

`learning_rate_init` indique la taille de chaque pas de gradient dans la méthode des gradients stochastiques.

`max_iter` donne le nombre maximal d'époques de descente de gradient pour l'apprentissage des poids.

On peut entrainer le MLP sur les données d'entrainement, puis calculer son score sur des données, par exemple sur les données de validation:

In [ ]:
mlp.fit(X_train, Y_train)

In [ ]:
mlp.score(X_val, Y_val)

Comment peut-on calculer le taux d'erreur du classeur à partir de ce score ?

### La validation simple

Un hyper-paramètre est un paramètre du classeur qui ne change pas au cours de l'apprentissage. Exemple : `learning_rate_init`, `hidden_layer_sizes`, `max_iter`.
On peut obtenir la liste complète en utilisant `get_params`.

In [ ]:
mlp.get_params()

On souhaite trouver les hyper-paramètres qui donnent le meilleur score pour le MLP entrainé. On va utiliser la méthode de "validation simple":
- On crée une liste de valeurs possibles pour les hyper-paramètres.
- Pour chaque valeur dans la liste,
    - on entraine (avec `fit`) un MLP avec ces valeurs d'hyper-paramètres sur les données d'entrainement,
    - on calcule le score de ce MLP sur les données de validation.
- On garde les valeurs d'hyper-paramètres qui donnent le meilleur score
- On évalue le MLP obtenu sur les données de test.

Utiliser la validation simple pour choisir la meilleure valeur de `learning_rate_init` parmi `[0.1, 0.01, 0.001, 0.0001]`, pour un MLP défini par
```
mlp = MLPClassifier(
    hidden_layer_sizes=(10,), activation="relu",
    solver="sgd", learning_rate="constant", learning_rate_init=lr,
    max_iter=2000, verbose=False)
```
où `lr` est la valeur de l'hyper-paramètre qu'on optimise.

- Afficher une courbe des scores de validation pour chaque valeur de `learning_rate_init`.

- Quel est le score du MLP choisi par la validation sur les données de test ?

Pour le classeur retenu après validation, tracer une courbe des erreurs d'entrainement et de test au fil de l'apprentissage en utilisant la fonction `partial_fit`. Attention: réinitialiser le mlp avant de procéder à l'apprentissage.

Utiliser la validation pour choisir à la fois `learning_rate_init` et le nombre de neurones dans la couche cachée du MLP (utiliser des petites valeurs pour le nombre de neurones, sinon le temps de calcul peut devenir très long).

### Validation croisée avec GridSearchCV

La validation croisée est une alternative à la validation simple, plus souvent utilisée.

On va utiliser l'objet `GridSearchCV` de scikit-learn.

On commence par définir les paramètres qu'on veut essayer:

In [ ]:
tuned_parameters = [{'learning_rate_init': [0.0001, 0.001, 0.01], 'hidden_layer_sizes': [(1,), (10,), (15,)],}]

On définit ensuite un classeur `GridSearchCV` à partir d'un MLP, de `tuned_parameters` et d'une fonction de score. Ici `accuracy` correspond au score qu'on a calculé précédemment.

Quand on appelle la fonction `fit` de ce classeur, le processus de validation est effectué complètement: toutes les valeurs des hyper-paramètres sont essayées, le score de chaque MLP est calculé et le meilleur modèle est retenu. On peut ensuite obtenir les meilleurs paramètres avec `clf.best_params_`. Le dictionnaire `clf.cv_results_` contient des informations sur le score de chaque MLP essayé.

Attention: ce processus peut être très long si `tuned_parameters` contient beaucoup de valeurs. Q: combien de MLP seront entrainés pour le tableau `tuned_parameters` ci-dessus?

In [ ]:
clf = GridSearchCV(
    MLPClassifier(activation="relu",
        solver="sgd", learning_rate="constant",
        max_iter=2000, verbose=False),
    tuned_parameters, scoring='accuracy')

Entrainer ce classeur et afficher pour chaque hyper-paramètre de la recherche le score du MLP obtenu.

Utiliser la fonction `classification_report(Y_test, Y_pred)` (où le deuxième argument contient les prédictions du classeur) pour examiner la performance du meilleur classeur obtenu. Que signifient les différentes colonnes ?

Question finale: trouver un bon classeur.